In [ ]:
# Installation
%%capture

!pip install transformers==4.5.1
!pip install datasets==1.6.2
!pip install tokenizers==0.10.2
!pip install torch==1.8.1+cu111
!pip install psutil==5.8.0
!pip install rouge_score
!pip install sacrebleu
!pip install openpyxl
!pip install xlrd
!pip install git-python
!pip install -U ipython==7.20
!pip install cmake
!pip install SentencePiece

In [ ]:
# Imports
import csv
import datasets
import gc
import psutil
import pandas as pd
import torch
import transformers

from datasets import ClassLabel
from IPython.display import display, HTML
from typing import Tuple

In [ ]:
# Drive
from google.colab import drive
drive.mount("/content/drive")
path_drive = "/content/drive/My Drive/Temp/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Config
language: str = "multilingual"  # english, german, multilingual
model_name: str = "xlm-roberta-base"
tokenizer_name: str = "xlm-roberta-base"
batch_size: int = 8

ratio_corpus_wik: float = 1.0
ratio_corpus_nws: float = 1.0
ratio_corpus_mls: float = 1.0
ratio_corpus_eng: float = 1.0

path_output: str = "/scratch/ws/1/davo557d-ws_project/"
path_checkpoint: str = "/content/drive/My Drive/Temp/Models/ML/XLM-R"

train_size: float = 0.900
val_size: float = 0.025
test_size: float = 0.075

corpus_german = [
    """ Die Täter wussten, was sie taten. Die Anschläge waren mediengerecht umgesetzt. Die Symbole, die sie zerstörten, waren präzise ausgewählt: Das weiße Haus und das Pentagon als Symbole der Macht, das World Trade Center als Symbol des Kapitals und der Wirtschaft. 2.750 Menschen, darunter 30 Deutsche, starben in oder an den Türmen des WTC, 147 in den Flugzeugen, 184 Opfer kamen im Pentagon ums Leben, 40 Menschen wurden in dem in Pennsylvania abgestürtzten Flugzeug Opfer des bisher perfidesten Terroranschlags, den islamistische Terroristen ausgelöst haben. Die USA befanden sich im Schockzustand: Die Anschläge waren der erste quasi militärische Angriff auf die Vereinigten Staaten seit 1814. Amerika erwies sich am 11. September 2001 erstmals seit den Kämpfen um seine Unabhängigkeit von England im späten 18. und frühen 19. Jahrhundert als im eigenen Land verwundbar. Die Anschläge hatten daher weltweit gravierende politische und militärische Folgen. Der amerikanische Präsident George W. Bush, der sich zum Zeitpunkt der Anschläge in Florida aufhielt, äußert sich in einer ersten Fernsehansprache: "Täuschen Sie sich nicht. Wir werden diese Leute bis zum Ende jagen und bestrafen." """,
    """ Bundestrainer Joachim Löw sah nach dem famosen 7:1-Triumph im Halbfinale gegen Gastgeber Brasilien keinen Anlass, seine Startelf umzustellen - bis kurz vor Anpfiff. Dann erreichte den Bundestrainer die Hiobsbotschaft: Khedira meldete sich mit Wadenproblemen kurzfristig ab, für ihn rückte Kramer in die Startformation. Das DFB-Team agierte durchaus bemüht, doch zielstrebiger blieben die Südamerikaner. Nach einer Flanke von Lavezzi schob Higuain zum vermeintlichen 1:0 ein, wurde aber wegen einer Abseitsposition korrekterweise zurückgepfiffen. Je länger die Partie dauerte, desto mehr scheuten beide Seiten das Risiko. Mit dem Mut schwand auch das Tempo aus der im ersten Abschnitt noch so rassigen Begegnung. Dafür wurden die Zweikämpfe ruppiger: Mascherano und Aguero kassierten Verwarnungen. Und wenn sich mal unverhofft eine Chance bot, schlich sich ein technischer Fehler ein. Es blieb ein zähes Ringen. Die bereits vorbelasteten Mascherano und Aguero hatten Glück, dass sie nicht vorzeitig mit Gelb-Rot vom Platz mussten, als sie Schweinsteiger erst umsensten und dann eine Platzwunde im Gesicht verschafften. Zwei ließ das kalt: Schürrle zog noch einen beherzten Spurt an und fand Götze im Zentrum, der das Spielgerät mit der Brust runterpflückte und gekonnt mit links volley einschoss - ein Traumtor zu einem ganz wichtigen Zeitpunkt. """,
    """ Die Inflation ist ein anhaltender Prozess der Geldentwertung, der sich durch allgemeine Preiserhöhungen bemerkbar macht. Mit einer Geldeinheit kann dann ständig weniger gekauft werden, d.h. die Kaufkraft des Geldes vermindert sich dauernd. Nicht als Inflation gelten einmalige, vorübergehende, durch ungewöhnliche Vorkommnisse verursachte Preisniveauerhöhungen sowie Preissteigerungen für bestimmte Güter oder Produktionsfaktoren. Die Inflation wird gemessen am Anstieg eines das allgemeine Preisniveau am besten widerspiegelnden Preisindexes wie z.B. des Verbraucherpreisindexes für Deutschland. Der prozentuale Anstieg des Preisindexes in einem bestimmten Zeitraum wird als Inflationsrate bezeichnet. Beim Entstehen einer Inflation spielt besonders die Geldmenge in der Volkswirtschaft eine große Rolle. Steht der gesamtwirtschaftlichen Gütermenge eine zu große Geldmenge gegenüber, ist eine Bedingung für die Inflation gegeben. Übersteigt die gesamtwirtschaftliche Güternachfrage das gesamtwirtschaftliche Güterangebot, das kurzfristig nicht erhöht werden kann, sind steigende Preise die Folge und die Inflation setzt ein. Die Preissteigerungen lösen steigende Löhne aus, wegen des höheren Einkommens steigt die Nachfrage nach Gütern an. Die höheren Löhne bewirken jedoch auch steigende Kosten der Unternehmen, was wiederum zu Preissteigerungen für Güter führt. """,
    """ Storniert ein Reisender wegen einer Virus-Pandemie die Reise, so steht dem Reiseveranstalter kein Anspruch auf Entschädigung gemäß § 651 h Abs. 1 Satz 3 BGB zu, wenn er ebenfalls die Reise absagt. Zudem begründet die fehlende Therapiemöglichkeit und Impfung ein kostenloses Reise­rücktritts­recht. Dies hat das Amtsgericht Stuttgart entschieden. Dem Fall lag folgender Sachverhalt zugrunde: Wenige Tage vor Beginn einer mehrtägigen Flug- und Busrundreise durch Portugal im März 2020 erklärte die Reisende die Stornierung. Sie begründete dies mit dem sich ausbreitenden Corona-Virus. Die Reiseveranstalterin sagte zwei Tage später ebenfalls die Reise ab. Dennoch verlangte sie von der Reisenden die Zahlung einer Stornogebühr. Da sich die Reisende weigerte dem nachzukommen, kam es zu einem Klageverfahren. Das Amtsgericht Stuttgart entschied zu Gunsten der Reisenden. Der Reiseveranstalterin stehe kein Anspruch auf Entschädigung gemäß § 651 h Abs. 1 Satz 3 BGB zu. Denn zum einen könne sich die Reisende auf außergewöhnliche Umstände im Sinne von § 651 h Abs. 3 BGB berufen. Zum anderen sei zu berücksichtigen, dass die Reiseveranstalterin ebenfalls die Reise abgesagt hat. """,
    """ Händewaschen ist eine der effektivsten Möglichkeiten, die Verbreitung von Keimen zu verhindern. Saubere Hände können die Ausbreitung von Keimen zwischen Personen und innerhalb einer ganzen Gemeinschaft verhindern. Befolgen Sie stets diese fünf Schritte. Befeuchten Sie Ihre Hände mit sauberem, fließendem Wasser (warm oder kalt), drehen Sie den Wasserhahn zu und verwenden Sie Seife. Schäumen Sie Ihre Hände ein, indem Sie sie mit der Seife aneinander reiben. Schäumen Sie die Handrücken, die Finger und die Fingernägel ein. Schrubben Sie Ihre Hände mindestens 20 Sekunden lang. Spülen Sie Ihre Hände gut unter fließendem, sauberem Wasser ab. Trocknen Sie Ihre Hände mit einem sauberen Handtuch ab oder lassen Sie sie an der Luft trocknen. """
]

corpus_english = [
    """ The perpetrators knew what they were doing. The attacks were media-ready. The symbols they destroyed were precisely chosen: The White House and the Pentagon as symbols of power, the World Trade Center as a symbol of capital and business. 2,750 people, including 30 Germans, died in or on the towers of the WTC, 147 in the airplanes, 184 victims lost their lives in the Pentagon, 40 people were killed in the plane that crashed in Pennsylvania in the most perfidious terrorist attack yet unleashed by Islamist terrorists. The U.S. was in a state of shock: The attacks were the first quasi-military attack on the United States since 1814. America proved vulnerable at home on Sept. 11, 2001, for the first time since the struggles for its independence from England in the late 18th and early 19th centuries. As a result, the attacks had serious political and military consequences around the world. U.S. President George W. Bush, who was in Florida at the time of the attacks, made the following remarks in an initial televised address: "Make no mistake. We will hunt these people down and punish them to the end." """,
    """ German coach Joachim Löw saw no reason to change his starting eleven after the fantastic 7-1 triumph in the semifinals against host Brazil - until shortly before kickoff. Then the coach received bad news: Khedira had to pull out at short notice with calf problems, and Kramer was replaced in the starting lineup. The DFB team tried hard, but the South Americans were more determined. After a cross from Lavezzi, Higuain put in what was supposed to be a 1-0, but was correctly whistled back for an offside position. The longer the game lasted, the more both sides shied away from taking risks. The tempo of the match, which had been so racy in the first period, also dwindled with the courage. Instead, the duels became rougher: Mascherano and Aguero received warnings. And whenever a chance presented itself unexpectedly, a technical error crept in. It remained a tough struggle. Mascherano and Aguero, who had already been penalized, were lucky not to be sent off early with a yellow card when they first knocked Schweinsteiger down and then gave him a cut to the face. Schürrle put in a spirited spurt and found Götze in the center, who picked the ball off his chest and expertly fired in a left-footed volley - a dream goal at a very important time. """,
    """ Inflation is a sustained process of demonetization, which manifests itself in general price increases. One monetary unit can then constantly buy less, i.e. the purchasing power of money is permanently reduced. Inflation does not include one-time, temporary price level increases caused by unusual occurrences or price increases for specific goods or production factors. Inflation is measured by the increase in a price index that best reflects the general price level, such as the consumer price index for Germany. The percentage increase in the price index over a given period is referred to as the inflation rate. The quantity of money in the economy plays a particularly important role in the occurrence of inflation. If the quantity of goods in the economy as a whole is matched by an excessively large quantity of money, a condition for inflation is met. If the aggregate demand for goods exceeds the aggregate supply of goods, which cannot be increased in the short term, rising prices are the consequence and inflation sets in. The price increases trigger rising wages, and because of the higher income, the demand for goods increases. However, the higher wages also cause companies' costs to rise, which in turn leads to price increases for goods. """,
    """ If a traveler cancels the trip due to a virus pandemic, the tour operator is not entitled to compensation under Section 651 h (1) sentence 3 of the German Civil Code (BGB) if the traveler also cancels the trip. In addition, the lack of treatment options and vaccination justifies a right to cancel the trip free of charge. This decided the district court Stuttgart. The case was based on the following facts: A few days before the start of a multi-day flight and bus tour through Portugal in March 2020, the traveler declared the cancellation. She justified this with the spreading Corona virus. The tour operator also canceled the trip two days later. Nevertheless, it demanded payment of a cancellation fee from the traveler. Since the traveler refused to comply, a lawsuit was filed. The Stuttgart District Court ruled in favor of the traveler. The tour operator was not entitled to compensation pursuant to Section 651 h (1) sentence 3 BGB. On the one hand, the traveler could invoke extraordinary circumstances within the meaning of Section 651 h (3) of the German Civil Code. Secondly, it had to be taken into account that the tour operator had also cancelled the trip. """,
    """ Washing your hands is easy, and it’s one of the most effective ways to prevent the spread of germs. Clean hands can stop germs from spreading from one person to another and throughout an entire community. Follow these five steps every time. Wet your hands with clean, running water (warm or cold), turn off the tap, and apply soap. Lather your hands by rubbing them together with the soap. Lather the backs of your hands, between your fingers, and under your nails. Scrub your hands for at least 20 seconds. Rinse your hands well under clean, running water. Dry your hands using a clean towel or air dry them. """
]

'''
- bert-base-multilingual-cased
- deepset/gbert-base
- xlm-roberta-base
- facebook/mbart-large-cc25
'''

'\n- bert-base-multilingual-cased\n- deepset/gbert-base\n- xlm-roberta-base\n- facebook/mbart-large-cc25\n'

In [ ]:
# Helpers
def load_data() -> Tuple[datasets.Dataset, datasets.Dataset, datasets.Dataset]:
    if language == "english":
        return load_english_data()

    if language == "german":
        return load_german_data()

    if language == "multilingual":
        return load_multilingual_data()


def load_english_data() -> Tuple[datasets.Dataset, datasets.Dataset, datasets.Dataset]:
    train_data = datasets.load_dataset(
        "cnn_dailymail", "3.0.0",
        split="train",
        ignore_verifications=True
    )

    val_data = datasets.load_dataset(
        "cnn_dailymail", "3.0.0",
        split="validation[:50%]",
        ignore_verifications=True
    )

    test_data = datasets.load_dataset(
        "cnn_dailymail", "3.0.0",
        split="test[:50%]",
        ignore_verifications=True
    )

    train_data = train_data.select(
        range(0, int(len(train_data) * ratio_corpus_eng))
    )

    train_data = train_data.rename_column("article", "text")
    train_data = train_data.rename_column("highlights", "summary")
    train_data = train_data.remove_columns("id")

    val_data = val_data.rename_column("article", "text")
    val_data = val_data.rename_column("highlights", "summary")
    val_data = val_data.remove_columns("id")

    test_data = test_data.rename_column("article", "text")
    test_data = test_data.rename_column("highlights", "summary")
    test_data = test_data.remove_columns("id")

    return train_data.shuffle(), val_data.shuffle(), test_data.shuffle()


def load_german_data() -> Tuple[datasets.Dataset, datasets.Dataset, datasets.Dataset]:
    ds_wik = load_corpus_wik()
    ds_nws = load_corpus_nws()
    ds_mls = load_corpus_mls()

    german_data = datasets.concatenate_datasets([
        ds_wik.select(
            range(0, int(len(ds_wik) * ratio_corpus_wik))),
        ds_nws.select(
            range(0, int(len(ds_nws) * ratio_corpus_nws))),
        ds_mls.select(
            range(0, int(len(ds_mls) * ratio_corpus_mls)))
    ])

    train_size = int(len(german_data) * 0.900)
    valid_size = int(len(german_data) * 0.025)
    test_size = int(len(german_data) * 0.075)

    train_data = german_data.select(
        range(0, train_size)
    )

    val_data = german_data.select(
        range(train_size, train_size + valid_size)
    )

    test_data = german_data.select(
        range(train_size + valid_size, train_size + valid_size + test_size)
    )

    return train_data, val_data, test_data


def load_corpus_wik() -> datasets.Dataset:
    data_txt, data_ref = [], []

    with open("/content/drive/My Drive/Temp/Corpus/data_train.csv", "r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter=",", quoting=csv.QUOTE_ALL)
        next(reader, None)

        for row in reader:
            data_txt.append(row[0])
            data_ref.append(row[1])

    df_wik = pd.DataFrame(
        list(zip(data_txt, data_ref)),
        columns=["text", "summary"]
    )

    ds_wik = datasets.arrow_dataset.Dataset.from_pandas(df_wik)

    return ds_wik.shuffle()


def load_corpus_nws() -> datasets.Dataset:
    df_nws = pd.read_excel("/content/drive/My Drive/Temp/Corpus/data_train_test.xlsx", engine="openpyxl")
    df_nws = df_nws[["article", "highlights"]]
    df_nws.columns = ["text", "summary"]
    df_nws = df_nws[~df_nws["summary"].str.contains("ZEIT")]
    df_nws = df_nws.dropna()
    ds_nws = datasets.arrow_dataset.Dataset.from_pandas(df_nws)
    ds_nws = ds_nws.remove_columns("__index_level_0__")

    return ds_nws.shuffle()


def load_corpus_mls() -> datasets.Dataset:
    ds_mls = datasets.load_dataset("mlsum", "de", split="train")
    ds_mls = ds_mls.remove_columns(["topic", "url", "title", "date"])

    text_corpus_mls = []
    summary_corpus_mls = []

    for entry in ds_mls:
        text = entry["text"]
        summary = entry["summary"]

        if summary in text:
            text = text[len(summary) + 1:len(text)]

        text_corpus_mls.append(text)
        summary_corpus_mls.append(summary)

    df_mls = pd.DataFrame(
        list(zip(text_corpus_mls, summary_corpus_mls)),
        columns=["text", "summary"]
    )

    ds_mls = datasets.arrow_dataset.Dataset.from_pandas(df_mls)

    return ds_mls.shuffle()


def load_multilingual_data() -> Tuple[datasets.Dataset, datasets.Dataset, datasets.Dataset]:
    english_data, _, _ = load_english_data()
    german_data, _, _ = load_german_data()

    multilingual_data = datasets.concatenate_datasets([
        german_data, english_data
    ]).shuffle()

    train_size = int(len(multilingual_data) * 0.900)
    valid_size = int(len(multilingual_data) * 0.025)
    test_size = int(len(multilingual_data) * 0.075)

    train_data = multilingual_data.select(
        range(0, train_size)
    )

    val_data = multilingual_data.select(
        range(train_size, train_size + valid_size)
    )

    test_data = multilingual_data.select(
        range(train_size + valid_size, train_size + valid_size + test_size)
    )

    return train_data, val_data, test_data


def test_cuda() -> None:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    torch.cuda.empty_cache()

    print("Device:", device)
    print("Version:", torch.__version__)


def explore_corpus(data: datasets.Dataset) -> None:
    df = pd.DataFrame(data)

    text_list = []
    summary_list = []

    for index, row in df.iterrows():
        text = row["text"]
        summary = row["summary"]
        text_list.append(len(text))
        summary_list.append(len(summary))

    df = pd.DataFrame(data[:1])

    for column, typ in data.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])


def empty_cache() -> None:
    gc.collect()
    torch.cuda.empty_cache()
    psutil.virtual_memory()


def load_tokenizer_and_model(from_checkpoint: bool = False) -> Tuple[transformers.AutoTokenizer, transformers.EncoderDecoderModel]:
    tokenizer = transformers.AutoTokenizer.from_pretrained(
        tokenizer_name, strip_accent=False
    )

    if from_checkpoint:
        if "mbart" in model_name:
            tf2tf = transformers.AutoModelForSeq2SeqLM.from_pretrained(
                path_checkpoint
            )

        else:
            tf2tf = transformers.EncoderDecoderModel.from_pretrained(
                path_checkpoint
            )

    else:
        if "mbart" in model_name:
            tf2tf = transformers.MBartForConditionalGeneration.from_pretrained(
                model_name
            )

        else:
            tf2tf = transformers.EncoderDecoderModel.from_encoder_decoder_pretrained(
                model_name, model_name, tie_encoder_decoder=True
            )

    return tokenizer, tf2tf


def configure_model(tf2tf: transformers.EncoderDecoderModel, tokenizer: transformers.AutoTokenizer):
    tf2tf.config.decoder_start_token_id = tokenizer.cls_token_id
    tf2tf.config.bos_token_id = tokenizer.bos_token_id
    tf2tf.config.eos_token_id = tokenizer.sep_token_id
    tf2tf.config.pad_token_id = tokenizer.pad_token_id

    tf2tf.config.max_length = 128
    tf2tf.config.min_length = 56
    tf2tf.config.no_repeat_ngram_size = 3
    tf2tf.config.early_stopping = True
    tf2tf.config.length_penalty = 2.0
    tf2tf.config.num_beams = 2

    return tf2tf


def split_long_texts(parts, text):
    limit = 512

    if len(text) > limit:
        end_index = max([
            text.rfind(".", 0, limit),
            text.rfind("!", 0, limit),
            text.rfind("?", 0, limit)
        ])

        parts.append(text[0:end_index + 1].strip())
        text = text[end_index + 1:len(text)].strip()
        parts = split_long_texts(parts, text)

    else:
        parts.append(text)

    return parts

In [ ]:
# Example (BERT, XLM-R)
tokenizer, tf2tf = load_tokenizer_and_model(from_checkpoint=True)

if language == "german":
    corpus = corpus_german

if language == "english":
    corpus = corpus_english

if language == "multilingual":
    corpus = corpus_english + corpus_german

tf2tf = configure_model(tf2tf, tokenizer)
tf2tf.to("cuda")

test_cuda()
empty_cache()

cnt = 0

for text in corpus:
    cnt += 1
    parts = split_long_texts([], text)

    if len(parts) > 1:
        article = parts
        highlights = [None] * len(parts)

    else:
        parts = [text]
        article = [text] * 2
        highlights = [None] * 2

    df = pd.DataFrame({"text": article, "summary": highlights})
    test_data = datasets.arrow_dataset.Dataset.from_pandas(df)


    def generate_summary(batch):
        inputs = tokenizer(
            batch["text"],
            padding="max_length",
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )

        input_ids = inputs.input_ids.to("cuda")
        attention_mask = inputs.attention_mask.to("cuda")

        outputs = tf2tf.generate(input_ids, attention_mask=attention_mask)
        output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        batch["pred_summary"] = output_str

        return batch


    summary = test_data.map(
        generate_summary,
        batched=True,
        batch_size=batch_size
    )

    result = ""

    for i in range(0, len(parts)):
        result = result + " " + summary[i]["pred_summary"]

    with open(path_output + "summary_" + language + "_" + model_name + "_text-" + str(cnt) + ".txt", "w", encoding="utf-8") as f:
        f.write(" ".join(parts))
        f.write("\n" + "====================" + "\n")
        f.write(summary[0]["pred_summary"])
        f.write("\n" + "====================" + "\n")
        f.write(result)


The following encoder weights were not tied to the decoder ['roberta/pooler']
The following encoder weights were not tied to the decoder ['roberta/pooler']


Device: cuda
Version: 1.8.1+cu101


In [ ]:
# Example (BART)
model = transformers.pipeline("summarization")
corpus = corpus_english + corpus_german

cnt = 0

for text in corpus:
    cnt += 1
    
    summary = model(
        text,
        max_length=156,
        min_length=64,
        do_sample=False
    )[0]["summary_text"]

    with open(path_output + "summary_" + language + "_BART_text-" + str(cnt) + ".txt", "w", encoding="utf-8") as f:
        f.write(summary)


Your max_length is set to 156, but you input_length is only 140. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


In [ ]:
# Evaluation (BART)
data, _, _ = load_data()
model = transformers.pipeline("summarization")

list_of_texts = list(data["text"])
list_of_candidates = list(data["summary"])
list_of_summaries, list_of_predictions = [], []

for i in range(0, 100):
    try:
        summary = model(
            str(list_of_texts[i])[0:1024],
            max_length=156,
            min_length=64,
            do_sample=False
        )[0]["summary_text"]
        
        list_of_predictions.append(summary)
        list_of_summaries.append(list_of_candidates[i])

    except Exception as e:
        print(e)

rouge = datasets.load_metric("rouge")

print(
    rouge.compute(
        predictions=list_of_predictions,
        references=list_of_summaries,
        rouge_types=["rouge2"]
    )["rouge2"].mid
)


Reusing dataset cnn_dailymail (/root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234)
Reusing dataset cnn_dailymail (/root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234)
Reusing dataset cnn_dailymail (/root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234)
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234/cache-1756de3f8e6e1baa.arrow
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234/cache-b36d67d6ef5694c0.arrow
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d4